In [3]:

import cv2
from ultralytics import YOLO
import threading
import subprocess
import torch
import os


### FLUX Youtube


In [12]:

class VideoStream:
    def __init__(self, src):
        self.cap = cv2.VideoCapture(src)
        self.ret, self.frame = self.cap.read()
        self.stopped = False

    def start(self):
        threading.Thread(target=self.update, args=()).start()
        return self

    def update(self):
        while not self.stopped:
            if not self.ret:
                self.stop()
            self.ret, self.frame = self.cap.read()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.cap.release()

# Charger le modèle YOLO
# model = YOLO("yolov8n.pt") #for Yolov8
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_best_fire_weights.pt')

# Remplacer VIDEO_ID par l'ID de la vidéo YouTube en direct
video_url = "https://www.youtube.com/watch?v=AGyccIEcsZ0"
command = ["yt-dlp", "-g", video_url]

try:
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
    url_m3u8 = result.stdout.strip()

    print("URL du flux m3u8:", url_m3u8)

    # Démarrer la capture vidéo avec threading
    video_stream = VideoStream(url_m3u8).start()

    window_width, window_height = 640, 360

    while True:
        frame = video_stream.read()

        if frame is None:
            print("Erreur : Impossible de lire le flux vidéo")
            break

        # Redimensionner la frame pour une meilleure fluidité
        frame_resized = cv2.resize(frame, (window_width, window_height))

        # Effectuer la détection d'objets sur la frame redimensionnée
        results = model(frame_resized)

        # for result in results:
        #     boxes = result.boxes  # Boîtes de délimitation
        #     if boxes is not None:
        #         for box in boxes:
        #             x1, y1, x2, y2 = box.xyxy[0]  # Coordonnées des coins
        #             conf = box.conf[0]  # Confiance
        #             if conf >= 0.3:  # Appliquer un seuil de confiance
        #                 cls = box.cls[0]  # Classe de l'objet détecté
        #                 label = f"{model.names[int(cls)]}: {conf:.2f}"
        #                 cv2.rectangle(frame_resized, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        #                 cv2.putText(frame_resized, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        for pred in results.pred[0]:
            x1, y1, x2, y2, conf, cls = pred[:6]
            if conf > 0.3:  # Appliquer un seuil de confiance
                label = f"{model.names[int(cls)]}: {conf:.2f}"
                cv2.rectangle(frame_resized, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame_resized, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Afficher la frame redimensionnée avec les détections
        cv2.imshow('Live Stream with YOLOv5 Detection', frame_resized)

        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_stream.stop()
    cv2.destroyAllWindows()

except subprocess.CalledProcessError as e:
    print(f"Erreur lors de l'exécution de la commande : {e}")
    print(f"Code de retour : {e.returncode}")
    print(f"Message d'erreur : {e.stderr}")


Using cache found in C:\Users\benfo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-24 Python-3.11.3 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


URL du flux m3u8: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1724539385/ei/mQ3KZpQ23OTE3w_g2IvQDA/ip/2a01:cb05:4c0:4100:f9ac:ebfb:eefe:bb51/id/AGyccIEcsZ0.1/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr5---sn-25glenlk.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/spc/Mv1m9sQxoSK_9KQrj-0x3aP8QceCVMYu73_s16GkbBhc30o233wXTTY/vprv/1/playlist_type/DVR/initcwndbps/1627500/mh/xR/mm/44/mn/sn-25glenlk/ms/lva/mv/m/mvi/5/pl/42/dover/11/pacing/0/keepalive/yes/mt/1724517643/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,spc,vprv,playlist_type/sig/AJfQdSswRQIgFsGiEk53n7h_gKz4MGTJij541fJQrv4nvlxRkmeNQo0CIQDeu7FIRPpKQsoRjsCYpCSD62XNvQHIBMhyVmPrg2IErg%3D%3D/lsparams/hls_chunk_host,initcwndbps,mh,mm,mn,ms,mv,mvi,pl/lsig/AGtxev0wRQIhAKQJuN84YiVUPdK1m0YRtO8SFjmzi2LMcSrRhpTbyXhYAiBX0xKyaZ9GKabDYUw_2p7sezMritkvajV_i5tqoXBIPg%3D%3D/playlist/in

Exception in thread Thread-19 (update):
Traceback (most recent call last):
  File "C:\Users\benfo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\benfo\OneDrive\Bureau\DjiProject\code\venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\benfo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\benfo\AppData\Local\Temp\ipykernel_12656\3111369610.py", line 15, in update
cv2.error: Unknown C++ exception from OpenCV code


### Image Local

In [10]:
# Répertoire contenant les images à inférer
image_dir = 'Images'  # Remplacez par le chemin de votre dossier d'images

# Répertoire de sortie pour les images avec détection (facultatif)
output_dir = os.path.expanduser('~/output_images')
os.makedirs(output_dir, exist_ok=True)

model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_best_fire_weights.pt')


# Boucle sur toutes les images du répertoire
for img_name in os.listdir(image_dir):
    # Charger l'image
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)

    # Effectuer la détection d'objets
    results = model(img)
    print("Start")
    # Afficher et sauvegarder les résultats
    for pred in results.pred[0]:
        print("pred")
    
        x1, y1, x2, y2, conf, cls = pred[:6]
       
        label = f"{model.names[int(cls)]}: {conf:.2f}"
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    
    # # Sauvegarder l'image avec les détections
    output_img_path = os.path.join(output_dir, img_name)
    cv2.imwrite(output_img_path, img)

    # Quitter avec la touche 'q'
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

# Fermer toutes les fenêtres
cv2.destroyAllWindows()

Using cache found in C:\Users\benfo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-24 Python-3.11.3 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Start
pred
Start
Start
Start
